In [7]:
# from roboflow import Roboflow 

# #Create object of roboflow 
# rf = Roboflow(api_key = "9HhgLNwnngYzUrANLRyF", model_format="multiclass") 
# dataset = rf.workspace().project("Dental Imaging").version(1).download(location="./dental_dataset")

Importing required libraries


In [8]:
import torch
from torch import nn
#base library for PyTorch computervision  
import torchvision 
from torchvision.transforms import ToTensor 




Creating dataset

In [9]:
import os
import pandas as pd
from torch.utils.data import Dataset


class dentalDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform

        csv_path = os.path.join(root_dir, "_classes.csv")
        self.df = pd.read_csv(csv_path)

        #filenames
        self.image_names = self.df.iloc[:, 0].values

        #class names from column
        self.class_names = list(self.df.columns[1:])

        #class index
        self.labels = self.df.iloc[:, 1:].values.argmax(axis=1)

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.image_names[idx])
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


Transforming dataset 

In [10]:
from torchvision import transforms

basicTransform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

Mapping datasets

In [11]:
train_data = dentalDataset(root_dir="dental_dataset/train",transform = basicTransform )

test_data = dentalDataset(root_dir="dental_dataset/test",transform=basicTransform)

validation_data = dentalDataset(root_dir="dental_dataset/test",transform = basicTransform)



Creating Dataloaders

In [12]:
from torch.utils.data import DataLoader
#setting up batch size 
BATCH_SIZE = 32

#shuffling data every epoch
train_dataloader = DataLoader(dataset = train_data, batch_size = BATCH_SIZE, shuffle = True)
test_dataloader = DataLoader(dataset = test_data, batch_size = BATCH_SIZE, shuffle = False)
val_dataloader = DataLoader(dataset = validation_data, batch_size = BATCH_SIZE, shuffle = True)
